## About
This notebook contains simple, toy examples to help you get started with FairMLHealth tool usage. This same content is mirrored in the repository's main [README](../README.md).

## Example Setup

In [1]:
from fairmlhealth import report as fhrp, measure, stat_utils

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


In [2]:
# First we'll create a semi-randomized dataframe with specific columns for our attributes of interest
np.random.seed(506)
N = 128
X = pd.DataFrame({'col1': np.random.randint(1, 4, N), 
                  'col2': np.random.randint(1, 75, N),
                  'col3': np.random.randint(0, 2, N),
                  'gender': [0, 1]*int(N/2), 
                  'ethnicity': [1, 1, 0, 0]*int(N/4),
                  'other': [1, 0, 0, 0, 1, 0, 0, 1]*int(N/8)
                 })

# Next we'll create a randomized target value with strong relationships to key features that will be 
# used in the examples below
#gender_corr = X['gender']
#gender_corr[(gender_corr.index+1)%4==0] = 0 # only associate with gender every other instance 
y = pd.Series(np.random.randint(0, 2, N), name='y')

# Third, we'll split the data and use it to train two generic models
splits = train_test_split(X, y, stratify=y, test_size=0.5, random_state=60)
X_train, X_test, y_train, y_test = splits

model_1 = BernoulliNB().fit(X_train, y_train)
model_2 = DecisionTreeClassifier().fit(X_train, y_train)



In [3]:
display(X.head(), y.head())

,col1,col2,col3,gender,ethnicity,other
0,1,53,0,0,1,1
1,3,51,0,1,1,0
2,1,23,1,0,0,0
3,2,1,0,1,0,0
4,1,68,0,0,1,1


0    1
1    1
2    1
3    0
4    0
Name: y, dtype: int64

## Model Measurement
The primary feature of this library is the model comparison tool. The current version supports assessment of binary prediction models through use of the measure_models and compare_models functions.

Measure_model is designed to generate a report of multiple fairness metrics for a single model. Here it is shown wrapped in a "flag" function to emphasize values that are outside of the "fair" range. 

In [4]:
# Generate a pandas dataframe of measures
fhrp.measure_model(X_test, y_test, X_test['gender'], model_1, flag_oor=True)

## Evaluation

FairMLHealth now also includes stratified table features to aid in identifying the source of unfairness or other bias: data tables, performance tables, and bias tables. Note that these stratified tables can evaluate multiple features at once, and that there are two options for identifying which features to assess.

Note that the flag tool has not yet been updated to work with stratified tables.

### Significance Testing


In [5]:
model_1_preds = pd.Series(model_1.predict(X_test))
display(model_1_preds.head())

gender_corr = X['gender']
#gender_corr[(gender_corr.index+1)%4!=0] = 0 # only associate with gender every other instance 
model_1_preds.loc[gender_corr.eq(1) & model_1_preds.eq(y_test)] = model_1_preds.map({0:1,1:0})
display(gender_corr.head())
display(model_1_preds.head())

pred_results = stat_utils.binary_result_labels(y_test, model_1_preds)
display(pred_results.head())

0    0
1    0
2    1
3    0
4    0
dtype: int64

0    0
1    1
2    0
3    1
4    0
Name: gender, dtype: int64

0    0
1    0
2    1
3    0
4    0
dtype: int64

0    TN
1    FN
2    TP
3    TN
4    TN
Name: prediction result, dtype: object

In [6]:
reject_h0 = stat_utils.bootstrap_significance(func=stat_utils.chisquare_pval, 
                                              group=X_test['gender'], 
                                              values=pred_results,
                                              n_samples=20)
print("Can we reject the hypothesis that prediction results are from the same distribution for each gender?\n", reject_h0)

Can we reject the hypothesis that prediction results are from the same distribution for each gender?
 False


In [7]:
reject_h0 = stat_utils.bootstrap_significance(func=stat_utils.chisquare_pval, 
                                              group=X_test['ethnicity'], 
                                              values=pred_results)
print("Can we reject the hypothesis that prediction results are from the same distribution for each ethnicity?\n", reject_h0)

Can we reject the hypothesis that prediction results are from the same distribution for each ethnicity?
 False


In [8]:
reject_h0 = stat_utils.bootstrap_significance(func=stat_utils.chisquare_pval, 
                                              group=X_test['other'], 
                                              values=pred_results)
print("Can we reject the hypothesis that prediction results are from the same distribution for each \"other\" category?\n", reject_h0)

Can we reject the hypothesis that prediction results are from the same distribution for each "other" category?
 False


### Stratified Data Tables

The data table is shown below with each of the two data argument options. It evaluates basic statistics specific to each feature-value, in addition to relative statistics for the target value. Since the table can be used to evaluate many features at once, it can be a useful option for identifying patterns of bias either alone or in concert with other (e.g., visual methods).

In [9]:
# Arguments Option 1: pass full set of data, subsetting with *features* argument
measure.data(X_test, y_test, features=['gender', 'other', 'col1'])

,Feature Name,Feature Value,Obs.,Entropy,Mean y,Median y,Missing Values,Std. Dev. y,Value Prevalence
0,ALL FEATURES,ALL VALUES,64,NaN,0.5312,1.0,0,0.5030,1.0000
1,gender,0,32,1.0000,0.4688,0.0,0,0.5070,0.5000
2,gender,1,32,1.0000,0.5938,1.0,0,0.4990,0.5000
3,other,0,38,0.9745,0.6316,1.0,0,0.4889,0.5938
4,other,1,26,0.9745,0.3846,0.0,0,0.4961,0.4062
5,col1,1,24,1.5693,0.5000,0.5,0,0.5108,0.3750
6,col1,2,23,1.5693,0.5652,1.0,0,0.5069,0.3594
7,col1,3,17,1.5693,0.5294,1.0,0,0.5145,0.2656


In [10]:
# Multiple targets can also be passed through the data table 
measure.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean col3,Median col2,Median col3,Missing Values,Std. Dev. col2,Std. Dev. col3,Value Prevalence
0,ALL FEATURES,ALL VALUES,64,NaN,42.7500,0.3438,46.5,0.0,0,21.6685,0.4787,1.0000
1,gender,0,32,1.0000,40.3438,0.3750,42.0,0.0,0,20.5859,0.4919,0.5000
2,gender,1,32,1.0000,45.1562,0.3125,53.0,0.0,0,22.7697,0.4709,0.5000
3,col1,1,24,1.5693,37.7917,0.2917,37.5,0.0,0,22.1477,0.4643,0.3750
4,col1,2,23,1.5693,42.5652,0.3478,43.0,0.0,0,21.6539,0.4870,0.3594
5,col1,3,17,1.5693,50.0000,0.4118,60.0,0.0,0,20.1525,0.5073,0.2656


In [11]:
# The "ALL FEATURES" overview column can be turned off via the add_overview argument
measure.data(X_test[['gender']], y_test, add_overview=False)

,Feature Name,Feature Value,Obs.,Entropy,Mean y,Median y,Missing Values,Std. Dev. y,Value Prevalence
0,gender,0,32,1.0,0.4688,0.0,0,0.507,0.5
1,gender,1,32,1.0,0.5938,1.0,0,0.499,0.5


In [12]:
# Multiple targets can also be passed through the data table 
measure.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'other'])

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean other,Median col2,Median other,Missing Values,Std. Dev. col2,Std. Dev. other,Value Prevalence
0,ALL FEATURES,ALL VALUES,64,NaN,42.7500,0.4062,46.5,0.0,0,21.6685,0.4950,1.0000
1,gender,0,32,1.0000,40.3438,0.5312,42.0,1.0,0,20.5859,0.5070,0.5000
2,gender,1,32,1.0000,45.1562,0.2812,53.0,0.0,0,22.7697,0.4568,0.5000
3,col1,1,24,1.5693,37.7917,0.4583,37.5,0.0,0,22.1477,0.5090,0.3750
4,col1,2,23,1.5693,42.5652,0.3043,43.0,0.0,0,21.6539,0.4705,0.3594
5,col1,3,17,1.5693,50.0000,0.4706,60.0,0.0,0,20.1525,0.5145,0.2656


### Stratified Performance Tables

The stratified performance table contains model performance measures specific to each feature-value subset. If prediction probabilities are available to the model, additional ROC AUC and PR AUC values will be included.

In [13]:
measure.performance(X_test[['gender']], y_test, model_1.predict(X_test), add_overview=False)

,Feature Name,Feature Value,Obs.,Accuracy,F1-Score,FPR,Mean __y_pred,Mean __y_true,Precision,TPR
0,gender,0,32.0,0.5000,0.5000,0.5294,0.5312,0.4688,0.4706,0.5333
1,gender,1,32.0,0.7188,0.7273,0.1538,0.4375,0.5938,0.8571,0.6316


In [14]:
measure.performance(X_test[['gender']], 
                    y_true=y_test, 
                    y_pred=model_1.predict(X_test), 
                    y_prob=model_1.predict_proba(X_test)[:,1])

,Feature Name,Feature Value,Obs.,Accuracy,F1-Score,FPR,Mean __y_pred,Mean __y_true,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,64.0,0.6094,0.6154,0.3667,0.4844,0.5312,NaN,0.6452,0.6069,0.5882
1,gender,0,32.0,0.5000,0.5000,0.5294,0.5312,0.4688,NaN,0.4706,0.5157,0.5333
2,gender,1,32.0,0.7188,0.7273,0.1538,0.4375,0.5938,NaN,0.8571,0.7470,0.6316


### Stratified Bias Fairness Tables

The stratified bias table contains model bias measures specific to each feature-value subset. Inspired by common measures of fairness, the tool assumes each feature-value as the "privileged" group relative to all other possible values for the feature. For example, row 3 in the table below displaying measures of "col1" value of "2" where 2 is considered to be the privileged group and all other values (1, 2, 45, and 50) are considered unprivileged.

To simplify the table, fairness measures have been reduced to their component parts. For example, measures of Equalized Odds can be determined by combining the True Positive Rate (TPR) Ratios & Differences with False Positive Rate (FPR) Ratios & Differences.

See also: [Fairness Quick References](../docs/Fairness_Quick_References.pdf) and the [Tutorial for Evaluating Fairness in Binary Classification](./Tutorial-EvaluatingFairnessInBinaryClassification.ipynb)

In [15]:
measure.bias(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), flag_oor=True)

,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,gender,0,0.2369,1.4720,-0.3756,0.2906,0.3866,1.8214,-0.0938,0.8235,0.0982,1.1842
1,gender,1,-0.2369,0.6794,0.3756,3.4412,-0.3866,0.5490,0.0938,1.2143,-0.0982,0.8444
2,col3,0,0.1000,1.1739,0.0500,1.1429,0.0812,1.1329,0.1623,1.3788,0.2500,1.5000
3,col3,1,-0.1000,0.8519,-0.0500,0.8750,-0.0812,0.8827,-0.1623,0.7253,-0.2500,0.6667


## Comparing Results for Multiple Models

The compare_models feature can be used to generate side-by-side fairness comparisons of multiple models. Model performance metrics such as accuracy and precision are also provided to facilitate comparison.   

Below is an example output comparing the two example models defined above. Missing values have been added for metrics requiring prediction probabilities, which the second model does not have (note the warning below).

In [18]:
# Pass the data and models to the compare models function, as above
fhrp.compare_models(X_test, y_test, X_test['gender'], 
                    {'model 1':model_1, 'model 2':model_2},
                    flag_oor=True)

The compare_models function can also be used to measure two different protected attributes. Protected attributes are measured separately and cannot yet be combined together with this tool.

In [19]:
fhrp.compare_models(X_test, y_test, 
                    [X_test['gender'], X_test['ethnicity']], 
                    {'gender':model_1, 'ethnicity':model_1},
                    flag_oor=True)


## Analysis by Cohort

In [29]:
measure.bias(X_test['col3'], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test['gender'])

,gender,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,col3,0,0.1338,1.2978,-0.0455,0.9167,0.1714,1.4286,0.0833,1.1667,0.2222,1.5000
1,0,col3,1,-0.1338,0.7706,0.0455,1.0909,-0.1714,0.7000,-0.0833,0.8571,-0.2222,0.6667
2,1,col3,0,0.0780,1.1093,0.1389,2.2500,-0.0417,0.9524,0.2364,1.6500,0.2949,1.5476
3,1,col3,1,-0.0780,0.9015,-0.1389,0.4444,0.0417,1.0500,-0.2364,0.6061,-0.2949,0.6462


In [31]:
measure.bias(X_test[['ethnicity', 'other']], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test[['col3', 'gender']])

/Users/christineallen/repos/fairMLHealth/fairmlhealth/__utils.py:55: UserWarning: Possible error in column(s) ['ethnicity']. Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
/Users/christineallen/repos/fairMLHealth/fairmlhealth/__utils.py:55: UserWarning: Possible error in column(s) ['other']. Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
/Users/christineallen/repos/fairMLHealth/fairmlhealth/__utils.py:55: UserWarning: Possible error in column(s) ['ethnicity']. Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
/Users/christineallen/repos/fairMLHealth/fairmlhealth/__utils.py:55: Us

AttributeError: 'int' object has no attribute 'lower'